Doubts from the paper:
1. Converting a white noise driven AR model to produce a Channel with accuracy comparable to SOS
2. Our first step is to use a high order AR model to generate samples close to the given sequence
3. Very unstable values when running with 12, 12 parameters, the value diverges quickly

To get a benchmark - plot out the frequency of retransmitting pilots - Fp, versus the Doppler frequency - Fd. 
The topmost curve is without using any modulation, just copying Jakes
Then if we use the previous channel to get the next channel value, then the curve will go down
The LSTM curve should be the lowest

Magnitude and phase - use it to do adaptive AR, no ARMA. 

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima_process import ArmaProcess
from scipy.signal import correlate
from scipy.linalg import toeplitz
from scipy.optimize import curve_fit

In [168]:
# Function for Sum of Sinusoids (SOS) method for Jakes channel generation
def jakes_sos(P, Fs, Fd, N, typ):
    t = np.linspace(0, P/Fs, P)
    omega_d = 2 * np.pi * Fd
    K = 1  # Number of random processes

    # Initialize jakes_rvs to store real or complex numbers
    jakes_rvs = np.zeros((K, P), dtype=complex)

    for k in range(K):
        alpha = np.random.uniform(0, 2 * np.pi, N)
        alpha_m = np.array([((2 * np.pi * n) - np.pi + al)/(4*N) for n, al in enumerate(alpha)])
        a_m = np.random.uniform(0, 2 * np.pi, N)
        b_m = np.random.uniform(0, 2 * np.pi, N)

        cosine_terms = np.cos((omega_d * t[:, None] * np.cos(alpha_m)) + a_m)
        real_part = np.sqrt(1/N) * np.sum(cosine_terms, axis=1)

        if typ == 'comp':
            sine_terms = np.sin((omega_d * t[:, None] * np.cos(alpha_m)) + b_m)
            imag_part = np.sqrt(1/N) * np.sum(sine_terms, axis=1)
            jakes_rvs[k] = real_part + 1j * imag_part
        else:
            jakes_rvs[k] = real_part + 1j * 0  # Enforce complex representation

    return jakes_rvs[0]  # Return the first (and only) process

In [170]:
# Function to create AR(P) model for Rayleigh fading
def create_ar_p_model(P, autocorr):
    # Use Yule-Walker equations to compute AR coefficients
    r = autocorr[:P+1]  # First P+1 lags of autocorrelation
    R = toeplitz(r[:-1])  # Create Toeplitz matrix
    ar_coeffs = np.linalg.solve(R, -r[1:])
    return ar_coeffs

# ARMA model equation
def arma_model(x, *params):
    p = len(params) // 2  # Half of the parameters for AR(p), half for MA(q)
    ar_params = np.r_[1, -np.array(params[:p])]  # AR parameters (with proper sign)
    ma_params = np.r_[1, np.array(params[p:])]   # MA parameters
    arma_process = ArmaProcess(ar_params, ma_params)
    return arma_process.generate_sample(nsample=len(x))

# Use least squares fitting to identify ARMA(p, q) coefficients with more evaluations and bounds
def arma_system_identification(ar_process_data, order_p, order_q):
    # Remove any NaNs or infinities from the data
    valid_data = np.isfinite(ar_process_data)
    x_data = np.arange(len(ar_process_data))[valid_data]  # Filter x indices
    y_data = ar_process_data[valid_data]                  # Filter valid y data

    # Initial guess for ARMA parameters (better guess: small random values or zeros)
    initial_guess = np.zeros(order_p + order_q)

    # Fit the ARMA model using least squares (curve_fit minimizes squared error)
    # Increase maxfev for more evaluations, and set bounds for the ARMA parameters
    optimal_params, _ = curve_fit(
        arma_model, x_data, y_data, p0=initial_guess, maxfev=20000,  # Increased maxfev
        bounds=(-10, 10)  # Constrain parameters within a reasonable range
    )

    # Separate AR and MA coefficients
    ar_params = optimal_params[:order_p]
    ma_params = optimal_params[order_p:]
    
    return ar_params, ma_params

In [172]:
# Plot autocorrelation comparison
def plot_autocorrelation(original_channel, arma_channel, sos_channel, Fs):
    lags = 1000  # Number of lags for autocorrelation
    autocorr_original = correlate(original_channel, original_channel, mode='full')
    autocorr_original = autocorr_original[autocorr_original.size // 2:]
    autocorr_original /= np.max(np.abs(autocorr_original))

    autocorr_arma = correlate(arma_channel, arma_channel, mode='full')
    autocorr_arma = autocorr_arma[autocorr_arma.size // 2:]
    autocorr_arma /= np.max(np.abs(autocorr_arma))

    autocorr_sos = correlate(sos_channel, sos_channel, mode='full')
    autocorr_sos = autocorr_sos[autocorr_sos.size // 2:]
    autocorr_sos /= np.max(np.abs(autocorr_sos))

    time_axis = np.arange(lags) / Fs

    plt.figure(figsize=(12, 6))
    plt.plot(time_axis[:lags], np.real(autocorr_original[:lags]), label='Original SOS Jakes Channel')
    plt.plot(time_axis[:lags], np.real(autocorr_arma[:lags]), label='ARMA Model Channel (12, 12)', linestyle='--')
    plt.plot(time_axis[:lags], np.real(autocorr_sos[:lags]), label='SOS Jakes Channel', linestyle=':')
    plt.title('Autocorrelation Functions Comparison')
    plt.xlabel('Time Lag (s)')
    plt.ylabel('Autocorrelation')
    plt.legend()
    plt.grid(True)
    plt.show()

In [174]:
# Function to insert original SOS values periodically
def insert_periodic_sos_values(arma_channel, sos_channel, period=100):
    arma_channel = np.copy(arma_channel)  # Avoid modifying original arma_channel in-place
    arma_channel[::period] = sos_channel[::period]  # Insert SOS values every 'period' samples
    return arma_channel

In [ ]:
# Main comparison function
def main():
    # Parameters
    P = 10000000  # Number of samples
    Fd = 100  # Doppler spread
    N = 100  # Number of sinusoids
    Fs = 10000  # Sampling frequency
    typ = 'comp'  # Complex channel

    # Generate Jakes fading channel using SOS method
    sos_channel = jakes_sos(P, Fs, Fd, N, typ)

    # Obtain autocorrelation sequence from the SOS method
    autocorr_sos = correlate(sos_channel, sos_channel, mode='full')[P - 1:]

    # Create AR(P) model (P = 50)
    P_order = 50
    ar_params = create_ar_p_model(P_order, autocorr_sos)

    # Generate AR(P) process using identified AR parameters
    arma_process = ArmaProcess(np.r_[1, -ar_params], np.r_[1])
    ar_process_data = arma_process.generate_sample(nsample=P)

    # Use least squares (curve_fit) to identify ARMA(12,12) model
    order_p, order_q = 12, 12
    ar_params, ma_params = arma_system_identification(ar_process_data, order_p, order_q)

    # Generate new ARMA(12,12) channel using the identified ARMA parameters
    arma_process = ArmaProcess(np.r_[1, -ar_params], np.r_[1, ma_params])
    arma_channel = arma_process.generate_sample(nsample=P)

    # Insert periodic SOS values into ARMA channel to stabilize it
    arma_channel_with_sos = insert_periodic_sos_values(arma_channel, sos_channel, period=100)

    # Plot and compare autocorrelation functions
    plot_autocorrelation(sos_channel, arma_channel_with_sos, sos_channel, Fs)
    
    # Plot magnitude comparison between SOS and ARMA(12,12) with SOS value insertion
    plt.figure(figsize=(12, 6))
    plt.plot(np.abs(sos_channel[:1000]), label='Original SOS Jakes Channel')
    plt.plot(np.abs(arma_channel_with_sos[:1000]), label='ARMA(12,12) with SOS Insertion', linestyle='--')
    plt.title('Channel Magnitude Comparison')
    plt.xlabel('Sample Index')
    plt.ylabel('Magnitude')
    plt.legend()
    plt.grid(True)
    plt.show()

if __name__ == "__main__":
    main()